In [1]:
import numpy as np
from imutils import paths
import cv2
import matplotlib.pyplot as plt
from sklearn import svm, datasets
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import classification_report,accuracy_score
import pandas as pd 
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow import keras
from sklearn.decomposition import PCA

In [2]:
path = ('C:/Users/borak/AMLS/dataset/image')

img_paths = sorted(list(paths.list_images(path)))

img_list = []

for path in img_paths:
    image_file = cv2.imread(path)
    gray = cv2.cvtColor(image_file, cv2.COLOR_BGR2GRAY)
    img_list.append(gray)

In [3]:
X = []
Y = []

labeldata = pd.read_csv ('C:/Users/borak/AMLS/dataset/label.csv')

for i in range (0,3000):
    X.append(img_list[i])
    if ("no_tumor") in (labeldata.iat[i,1]):
        Y.append(0)
    elif ("meningioma_tumor") in (labeldata.iat[i,1]):
        Y.append(1)
    elif ("glioma_tumor") in (labeldata.iat[i,1]):
        Y.append(2)
    elif ("pituitary_tumor") in (labeldata.iat[i,1]):
        Y.append(3)

In [4]:
X=np.array(X) 
X_resized=X.flatten().reshape(len(X),-1) 
X_resized.shape 

(3000, 262144)

In [5]:
Y=np.array(Y) 
Y_resized=Y.flatten().reshape(len(Y),-1) 
Y_resized.shape 

(3000, 1)

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X_resized, Y_resized, random_state = 42, test_size=0.8)

In [44]:
def allocate_weights_and_biases():

    # define number of hidden layers ..
    n_hidden_1 = 2048  # 1st layer number of neurons
    n_hidden_2 = 2048  # 2nd layer number of neurons

    # inputs placeholders
    X_resized = tf.placeholder("float", [None, 68, 2])
    Y_resized = tf.placeholder("float", [None, 2])  # 2 output classes  
    # flatten image features into one vector (i.e. reshape image feature matrix into a vector)
#images_flat = tf.contrib.layers.flatten(X)  
    # weights and biases are initialized from a normal distribution with a specified standard devation stddev
    stddev = 0.01
    weights = {
        'hidden_layer1': tf.Variable(tf.random.normal([68 * 2, 2048], stddev=stddev)),
        'hidden_layer2': tf.Variable(tf.random.normal([2048, 2048], stddev=stddev)),
        'out': tf.Variable(tf.random.normal([2048, 2], stddev=stddev))
    }

    biases = {
        'bias_layer1': tf.Variable(tf.random.normal([2048], stddev=stddev)),
        'bias_layer2': tf.Variable(tf.random.normal([2048], stddev=stddev)),
        'out': tf.Variable(tf.random.normal([2], stddev=stddev))
    }
    return weights, biases, X, Y, images_flat


In [45]:
# Create model
def multilayer_perceptron():
        
    weights, biases, X, Y, images_flat = allocate_weights_and_biases()

    # Hidden fully connected layer 1
    layer_1 = tf.add(tf.matmul(images_flat, weights['hidden_layer1']), biases['bias_layer1'])
    layer_1 = tf.sigmoid(layer_1)

    # Hidden fully connected layer 2
    layer_2 = tf.add(tf.matmul(layer_1, weights['hidden_layer2']), biases['bias_layer2'])
    layer_2 = tf.sigmoid(layer_2)
    
    # Output fully connected layer
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']

    return out_layer, X, Y


In [41]:
# learning parameters
learning_rate = 1e-5
training_epochs = 500

# display training accuracy every ..
display_accuracy_step = 10

    
#training_images, training_labels, test_images, test_labels = get_data()
logits, X, Y = multilayer_perceptron()

# define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)

# define training graph operation
train_op = optimizer.minimize(loss_op)

# graph operation to initialize all variables
init_op = tf.global_variables_initializer()

RuntimeError: tf.placeholder() is not compatible with eager execution.